In [70]:
import selenium
from bs4 import BeautifulSoup
import requests
from tqdm.notebook import tqdm
import pandas as pd

pageNumber = 1
base_url= f'https://m.bobaedream.co.kr/mycar/d_search_list/{pageNumber}?sel_m_gubun=&gubun=A&maker_no=&model_no=&group_no=&buy_year1=&buy_month1=&buy_year2=&buy_month2=&car_color=&car_option=&price1=100&price2=&min_km=&max_km=&chk-min-years=on&chk-min-month=on&chk-max-years=on&chk-max-month=on&lease_mprice1=&lease_mprice2=&lease_rmonth1=&lease_rmonth2=&lease_tprice1=&lease_tprice2=&l_year1=&l_year2='
resp = requests.get(base_url)
soup = BeautifulSoup(resp.content)

# 마지막 페이지 가져오기 
a_tags = soup.select('a.last')
mlist = a_tags[0]['href'].split('/')
temp = mlist[3].split('?')
last_page = int(temp[0])

accidentKeys = ['total loss', 'robbery','submerged','special_use', 'my_damage','opponent_damage','owner_changes','number_changes', 'rental_use','business_use','official_use']
repairKeys = ['도장', '탈착', '교환','판금','수리','기타']
total_columns = accidentKeys.extend(repairKeys)

df = pd.DataFrame(columns=total_columns)

for pageNumber in tqdm(range(1,last_page+1)) :
    resp = requests.get(base_url)
    soup = BeautifulSoup(resp.content)

    a_tags = soup.select('div.car-description > a.link')
    linkList = []
   
    for a_tag in a_tags :
        linkList.append(a_tag['href'])

    url = 'https://m.bobaedream.co.kr/'
    

    for link in linkList : 
        resp = requests.get(url+link)
        soup = BeautifulSoup(resp.content)

        car_name = soup.select('h1.tit-area')[0].text
        cno = link.split('/')[3]
        history_url = f'https://m.bobaedream.co.kr/proc/mycarChart_B.php?car_number={car_name}&tbl=mycar&cno={cno}'
        
        resp = requests.get(history_url)
        soup = BeautifulSoup(resp.content)


        div_tags = soup.select('div.price')
        
        accidentDict = {}
        if len(div_tags) != 0 :
            accidentValues = []
            for div in div_tags[:11] :
                value = div.text.strip('\n\t ') 
                #value = 0 if value == '없음' else value.replace('회','').replace('(','').replace('원)','').replace(',','')
                accidentValues.append(value)
            for k, v in zip(accidentKeys,accidentValues) :
                accidentDict[k] = v

            # if accidentDict['my_damage'] != 0 :
            #     my_damage = accidentDict['my_damage'].split(' ')
            #     accidentDict['my_damage_cnt'] = my_damage[0]
            #     accidentDict['my_damage_price'] = my_damage[1]
            # else : 
            #     accidentDict['my_damage_cnt'] = 0
            #     accidentDict['my_damage_price'] = 0

            # if accidentDict['opponent_damage'] != 0 :
            #     opponent_damage = accidentDict['opponent_damage'].split(' ')
            #     accidentDict['opponent_damage_cnt_damage_cnt'] = opponent_damage[0]
            #     accidentDict['opponent_damage_price'] = opponent_damage[1]
            # else : 
            #     accidentDict['opponent_damage_cnt_damage_cnt'] = 0
            #     accidentDict['opponent_damage_price'] = 0

            # accidentDict.pop('my_damage')
            # accidentDict.pop('opponent_damage')
        else :
            for k in accidentKeys :
                accidentDict[k] = None

        repairDict = {}
        
        repairValues = []
        div_tags = soup.select('div#repairAll div.td')
        if len(div_tags)==0 :
            for k in repairKeys :
                repairDict[k] = None
        else :
            cntIdx = 2
            for div in div_tags :
                repairValues.append(int(div.text.split(' ')[cntIdx]))
            for idx in range(len(repairKeys)) :
                repairDict[repairKeys[idx]] = repairValues[idx]

        total_dict = {}
        total_dict['cno'] = cno
        for accident_key in accidentDict.keys() :
            total_dict[accident_key] = accidentDict[accident_key]
        for repair_key in repairDict.keys() :
            total_dict[repair_key] = repairDict[repair_key]

        temp_df = pd.DataFrame(total_dict, index = [0])
        df = pd.concat([df, temp_df], ignore_index=True)


      

  0%|          | 0/47 [00:00<?, ?it/s]

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHA

In [75]:
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2350 entries, 0 to 2349
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   cno              2350 non-null   object
 1   total loss       2350 non-null   object
 2   robbery          2350 non-null   object
 3   submerged        2350 non-null   object
 4   special_use      2350 non-null   object
 5   my_damage        2350 non-null   object
 6   opponent_damage  2350 non-null   object
 7   owner_changes    2350 non-null   object
 8   number_changes   2350 non-null   object
 9   rental_use       2350 non-null   object
 10  business_use     2350 non-null   object
 11  official_use     2350 non-null   object
 12  도장               706 non-null    object
 13  탈착               706 non-null    object
 14  교환               706 non-null    object
 15  판금               706 non-null    object
 16  수리               706 non-null    object
 17  기타               706 non-null    

In [74]:
df.to_csv('insurance_history', index=False, encoding='utf-8')

In [53]:
df.drop(df[df['total loss'] == ''].index, inplace=True)

In [64]:
print(df['total loss'].unique())
print(df['robbery'].unique())

# df[df['submerged']=='있음']= True
# df[df['special_use']=='있음']= True
# df[df['rental_use']=='있음'] = True
# df[df['business_use']=='있음'] = True
# df[df['official_use']=='있음'] = True

# df[df['submerged']==0] = False
# df[df['special_use']==0] = False
# df[df['rental_use']==0] = False
# df[df['business_use']==0] = False
# df[df['official_use']==0] = False

[0]
[0]


C:\Users\user\AppData\Local\Temp\ipykernel_12688\3285545782.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'True' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df[df['special_use']=='있음']= True
C:\Users\user\AppData\Local\Temp\ipykernel_12688\3285545782.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'True' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df[df['special_use']=='있음']= True
C:\Users\user\AppData\Local\Temp\ipykernel_12688\3285545782.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'True' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df[df['special_use']=='있음']= True


[False True]
[False True]
[False True]
[False True]
[False True]
[False True]
[False True]
